In [24]:
import pandas as pd
import tensorflow as tf

TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path

def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    train_path, test_path = maybe_download()

    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)


(train_x, train_y), (test_x, test_y) = load_data()

print(train_x.shape)
print(train_y.shape)

(120, 4)
(120,)


In [25]:
#print(type(train_x))
print(train_x.head())

   SepalLength  SepalWidth  PetalLength  PetalWidth
0          6.4         2.8          5.6         2.2
1          5.0         2.3          3.3         1.0
2          4.9         2.5          4.5         1.7
3          4.9         3.1          1.5         0.1
4          5.7         3.8          1.7         0.3


#### Panda DataFrame to Numpy arrays

In [26]:
import numpy as np
#train_x_np = train_x.as_matrix(columns=None) # Depricated
train_x_np = train_x.values
print(train_x_np)

[[6.4 2.8 5.6 2.2]
 [5.  2.3 3.3 1. ]
 [4.9 2.5 4.5 1.7]
 [4.9 3.1 1.5 0.1]
 [5.7 3.8 1.7 0.3]
 [4.4 3.2 1.3 0.2]
 [5.4 3.4 1.5 0.4]
 [6.9 3.1 5.1 2.3]
 [6.7 3.1 4.4 1.4]
 [5.1 3.7 1.5 0.4]
 [5.2 2.7 3.9 1.4]
 [6.9 3.1 4.9 1.5]
 [5.8 4.  1.2 0.2]
 [5.4 3.9 1.7 0.4]
 [7.7 3.8 6.7 2.2]
 [6.3 3.3 4.7 1.6]
 [6.8 3.2 5.9 2.3]
 [7.6 3.  6.6 2.1]
 [6.4 3.2 5.3 2.3]
 [5.7 4.4 1.5 0.4]
 [6.7 3.3 5.7 2.1]
 [6.4 2.8 5.6 2.1]
 [5.4 3.9 1.3 0.4]
 [6.1 2.6 5.6 1.4]
 [7.2 3.  5.8 1.6]
 [5.2 3.5 1.5 0.2]
 [5.8 2.6 4.  1.2]
 [5.9 3.  5.1 1.8]
 [5.4 3.  4.5 1.5]
 [6.7 3.  5.  1.7]
 [6.3 2.3 4.4 1.3]
 [5.1 2.5 3.  1.1]
 [6.4 3.2 4.5 1.5]
 [6.8 3.  5.5 2.1]
 [6.2 2.8 4.8 1.8]
 [6.9 3.2 5.7 2.3]
 [6.5 3.2 5.1 2. ]
 [5.8 2.8 5.1 2.4]
 [5.1 3.8 1.5 0.3]
 [4.8 3.  1.4 0.3]
 [7.9 3.8 6.4 2. ]
 [5.8 2.7 5.1 1.9]
 [6.7 3.  5.2 2.3]
 [5.1 3.8 1.9 0.4]
 [4.7 3.2 1.6 0.2]
 [6.  2.2 5.  1.5]
 [4.8 3.4 1.6 0.2]
 [7.7 2.6 6.9 2.3]
 [4.6 3.6 1.  0.2]
 [7.2 3.2 6.  1.8]
 [5.  3.3 1.4 0.2]
 [6.6 3.  4.4 1.4]
 [6.1 2.8 4.

#### To Tensorflow Dataset

In [27]:
train_tf_ds = tf.data.Dataset.from_tensor_slices(train_x_np)

<BatchDataset shapes: (?, 4), types: tf.float64>

In [28]:
print(train_tf_ds.output_types)  # ==> "{'a': tf.float32, 'b': tf.int32}"
print(train_tf_ds.output_shapes)  # ==> "{'a': (), 'b': (100,)}"

<dtype: 'float64'>
(4,)


#### Batch the Training Dataset

In [39]:
train_tf_ds_rpt = train_tf_ds.repeat(10)
train_tf_ds_btc = train_tf_ds_rpt.batch(5)


#### Run an iterator on the Dataset

In [42]:
#iterator  = train_tf_ds_btc.make_one_shot_iterator()
iterator = train_tf_ds_btc.make_initializable_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    for i in range(10):
        sess.run(iterator.initializer)
        while True:
          try:
            print(sess.run(next_element)) # output [ 0.52374458  0.71968478]
          except tf.errors.OutOfRangeError:
            break
        print("Epoch ", i)

[[6.4 2.8 5.6 2.2]
 [5.  2.3 3.3 1. ]
 [4.9 2.5 4.5 1.7]
 [4.9 3.1 1.5 0.1]
 [5.7 3.8 1.7 0.3]]
[[4.4 3.2 1.3 0.2]
 [5.4 3.4 1.5 0.4]
 [6.9 3.1 5.1 2.3]
 [6.7 3.1 4.4 1.4]
 [5.1 3.7 1.5 0.4]]
[[5.2 2.7 3.9 1.4]
 [6.9 3.1 4.9 1.5]
 [5.8 4.  1.2 0.2]
 [5.4 3.9 1.7 0.4]
 [7.7 3.8 6.7 2.2]]
[[6.3 3.3 4.7 1.6]
 [6.8 3.2 5.9 2.3]
 [7.6 3.  6.6 2.1]
 [6.4 3.2 5.3 2.3]
 [5.7 4.4 1.5 0.4]]
[[6.7 3.3 5.7 2.1]
 [6.4 2.8 5.6 2.1]
 [5.4 3.9 1.3 0.4]
 [6.1 2.6 5.6 1.4]
 [7.2 3.  5.8 1.6]]
[[5.2 3.5 1.5 0.2]
 [5.8 2.6 4.  1.2]
 [5.9 3.  5.1 1.8]
 [5.4 3.  4.5 1.5]
 [6.7 3.  5.  1.7]]
[[6.3 2.3 4.4 1.3]
 [5.1 2.5 3.  1.1]
 [6.4 3.2 4.5 1.5]
 [6.8 3.  5.5 2.1]
 [6.2 2.8 4.8 1.8]]
[[6.9 3.2 5.7 2.3]
 [6.5 3.2 5.1 2. ]
 [5.8 2.8 5.1 2.4]
 [5.1 3.8 1.5 0.3]
 [4.8 3.  1.4 0.3]]
[[7.9 3.8 6.4 2. ]
 [5.8 2.7 5.1 1.9]
 [6.7 3.  5.2 2.3]
 [5.1 3.8 1.9 0.4]
 [4.7 3.2 1.6 0.2]]
[[6.  2.2 5.  1.5]
 [4.8 3.4 1.6 0.2]
 [7.7 2.6 6.9 2.3]
 [4.6 3.6 1.  0.2]
 [7.2 3.2 6.  1.8]]
[[5.  3.3 1.4 0.2]
 [6.6 3.  4.4 1.4]
 [

[[4.6 3.4 1.4 0.3]
 [4.6 3.2 1.4 0.2]
 [7.7 2.8 6.7 2. ]
 [5.9 3.2 4.8 1.8]
 [5.1 3.8 1.6 0.2]]
[[4.9 3.  1.4 0.2]
 [4.9 2.4 3.3 1. ]
 [4.5 2.3 1.3 0.3]
 [5.8 2.7 4.1 1. ]
 [5.  3.4 1.6 0.4]]
[[5.2 3.4 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.6 2.9 3.6 1.3]
 [4.8 3.1 1.6 0.2]]
[[6.3 2.7 4.9 1.8]
 [5.7 2.8 4.1 1.3]
 [5.  3.  1.6 0.2]
 [6.3 3.3 6.  2.5]
 [5.  3.5 1.6 0.6]]
[[5.5 2.6 4.4 1.2]
 [5.7 3.  4.2 1.2]
 [4.4 2.9 1.4 0.2]
 [4.8 3.  1.4 0.1]
 [5.5 2.4 3.7 1. ]]
[[6.4 2.8 5.6 2.2]
 [5.  2.3 3.3 1. ]
 [4.9 2.5 4.5 1.7]
 [4.9 3.1 1.5 0.1]
 [5.7 3.8 1.7 0.3]]
[[4.4 3.2 1.3 0.2]
 [5.4 3.4 1.5 0.4]
 [6.9 3.1 5.1 2.3]
 [6.7 3.1 4.4 1.4]
 [5.1 3.7 1.5 0.4]]
[[5.2 2.7 3.9 1.4]
 [6.9 3.1 4.9 1.5]
 [5.8 4.  1.2 0.2]
 [5.4 3.9 1.7 0.4]
 [7.7 3.8 6.7 2.2]]
[[6.3 3.3 4.7 1.6]
 [6.8 3.2 5.9 2.3]
 [7.6 3.  6.6 2.1]
 [6.4 3.2 5.3 2.3]
 [5.7 4.4 1.5 0.4]]
[[6.7 3.3 5.7 2.1]
 [6.4 2.8 5.6 2.1]
 [5.4 3.9 1.3 0.4]
 [6.1 2.6 5.6 1.4]
 [7.2 3.  5.8 1.6]]
[[5.2 3.5 1.5 0.2]
 [5.8 2.6 4.  1.2]
 [

 [7.  3.2 4.7 1.4]]
[[6.  3.  4.8 1.8]
 [7.4 2.8 6.1 1.9]
 [5.8 2.7 5.1 1.9]
 [6.2 3.4 5.4 2.3]
 [5.  2.  3.5 1. ]]
[[5.6 2.5 3.9 1.1]
 [6.7 3.1 5.6 2.4]
 [6.3 2.5 5.  1.9]
 [6.4 3.1 5.5 1.8]
 [6.2 2.2 4.5 1.5]]
[[7.3 2.9 6.3 1.8]
 [4.4 3.  1.3 0.2]
 [7.2 3.6 6.1 2.5]
 [6.5 3.  5.5 1.8]
 [5.  3.4 1.5 0.2]]
[[4.7 3.2 1.3 0.2]
 [6.6 2.9 4.6 1.3]
 [5.5 3.5 1.3 0.2]
 [7.7 3.  6.1 2.3]
 [6.1 3.  4.9 1.8]]
[[4.9 3.1 1.5 0.1]
 [5.5 2.4 3.8 1.1]
 [5.7 2.9 4.2 1.3]
 [6.  2.9 4.5 1.5]
 [6.4 2.7 5.3 1.9]]
[[5.4 3.7 1.5 0.2]
 [6.1 2.9 4.7 1.4]
 [6.5 2.8 4.6 1.5]
 [5.6 2.7 4.2 1.3]
 [6.3 3.4 5.6 2.4]]
[[4.9 3.1 1.5 0.1]
 [6.8 2.8 4.8 1.4]
 [5.7 2.8 4.5 1.3]
 [6.  2.7 5.1 1.6]
 [5.  3.5 1.3 0.3]]
[[6.5 3.  5.2 2. ]
 [6.1 2.8 4.7 1.2]
 [5.1 3.5 1.4 0.3]
 [4.6 3.1 1.5 0.2]
 [6.5 3.  5.8 2.2]]
[[4.6 3.4 1.4 0.3]
 [4.6 3.2 1.4 0.2]
 [7.7 2.8 6.7 2. ]
 [5.9 3.2 4.8 1.8]
 [5.1 3.8 1.6 0.2]]
[[4.9 3.  1.4 0.2]
 [4.9 2.4 3.3 1. ]
 [4.5 2.3 1.3 0.3]
 [5.8 2.7 4.1 1. ]
 [5.  3.4 1.6 0.4]]
[[5.2 3.4 1.4 0.2]
 

[[7.3 2.9 6.3 1.8]
 [4.4 3.  1.3 0.2]
 [7.2 3.6 6.1 2.5]
 [6.5 3.  5.5 1.8]
 [5.  3.4 1.5 0.2]]
[[4.7 3.2 1.3 0.2]
 [6.6 2.9 4.6 1.3]
 [5.5 3.5 1.3 0.2]
 [7.7 3.  6.1 2.3]
 [6.1 3.  4.9 1.8]]
[[4.9 3.1 1.5 0.1]
 [5.5 2.4 3.8 1.1]
 [5.7 2.9 4.2 1.3]
 [6.  2.9 4.5 1.5]
 [6.4 2.7 5.3 1.9]]
[[5.4 3.7 1.5 0.2]
 [6.1 2.9 4.7 1.4]
 [6.5 2.8 4.6 1.5]
 [5.6 2.7 4.2 1.3]
 [6.3 3.4 5.6 2.4]]
[[4.9 3.1 1.5 0.1]
 [6.8 2.8 4.8 1.4]
 [5.7 2.8 4.5 1.3]
 [6.  2.7 5.1 1.6]
 [5.  3.5 1.3 0.3]]
[[6.5 3.  5.2 2. ]
 [6.1 2.8 4.7 1.2]
 [5.1 3.5 1.4 0.3]
 [4.6 3.1 1.5 0.2]
 [6.5 3.  5.8 2.2]]
[[4.6 3.4 1.4 0.3]
 [4.6 3.2 1.4 0.2]
 [7.7 2.8 6.7 2. ]
 [5.9 3.2 4.8 1.8]
 [5.1 3.8 1.6 0.2]]
[[4.9 3.  1.4 0.2]
 [4.9 2.4 3.3 1. ]
 [4.5 2.3 1.3 0.3]
 [5.8 2.7 4.1 1. ]
 [5.  3.4 1.6 0.4]]
[[5.2 3.4 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.6 2.9 3.6 1.3]
 [4.8 3.1 1.6 0.2]]
[[6.3 2.7 4.9 1.8]
 [5.7 2.8 4.1 1.3]
 [5.  3.  1.6 0.2]
 [6.3 3.3 6.  2.5]
 [5.  3.5 1.6 0.6]]
[[5.5 2.6 4.4 1.2]
 [5.7 3.  4.2 1.2]
 [

[[6.3 2.7 4.9 1.8]
 [5.7 2.8 4.1 1.3]
 [5.  3.  1.6 0.2]
 [6.3 3.3 6.  2.5]
 [5.  3.5 1.6 0.6]]
[[5.5 2.6 4.4 1.2]
 [5.7 3.  4.2 1.2]
 [4.4 2.9 1.4 0.2]
 [4.8 3.  1.4 0.1]
 [5.5 2.4 3.7 1. ]]
[[6.4 2.8 5.6 2.2]
 [5.  2.3 3.3 1. ]
 [4.9 2.5 4.5 1.7]
 [4.9 3.1 1.5 0.1]
 [5.7 3.8 1.7 0.3]]
[[4.4 3.2 1.3 0.2]
 [5.4 3.4 1.5 0.4]
 [6.9 3.1 5.1 2.3]
 [6.7 3.1 4.4 1.4]
 [5.1 3.7 1.5 0.4]]
[[5.2 2.7 3.9 1.4]
 [6.9 3.1 4.9 1.5]
 [5.8 4.  1.2 0.2]
 [5.4 3.9 1.7 0.4]
 [7.7 3.8 6.7 2.2]]
[[6.3 3.3 4.7 1.6]
 [6.8 3.2 5.9 2.3]
 [7.6 3.  6.6 2.1]
 [6.4 3.2 5.3 2.3]
 [5.7 4.4 1.5 0.4]]
[[6.7 3.3 5.7 2.1]
 [6.4 2.8 5.6 2.1]
 [5.4 3.9 1.3 0.4]
 [6.1 2.6 5.6 1.4]
 [7.2 3.  5.8 1.6]]
[[5.2 3.5 1.5 0.2]
 [5.8 2.6 4.  1.2]
 [5.9 3.  5.1 1.8]
 [5.4 3.  4.5 1.5]
 [6.7 3.  5.  1.7]]
[[6.3 2.3 4.4 1.3]
 [5.1 2.5 3.  1.1]
 [6.4 3.2 4.5 1.5]
 [6.8 3.  5.5 2.1]
 [6.2 2.8 4.8 1.8]]
[[6.9 3.2 5.7 2.3]
 [6.5 3.2 5.1 2. ]
 [5.8 2.8 5.1 2.4]
 [5.1 3.8 1.5 0.3]
 [4.8 3.  1.4 0.3]]
[[7.9 3.8 6.4 2. ]
 [5.8 2.7 5.1 1.9]
 [

...

[[6.3 2.3 4.4 1.3]
 [5.1 2.5 3.  1.1]
 [6.4 3.2 4.5 1.5]
 [6.8 3.  5.5 2.1]
 [6.2 2.8 4.8 1.8]]
[[6.9 3.2 5.7 2.3]
 [6.5 3.2 5.1 2. ]
 [5.8 2.8 5.1 2.4]
 [5.1 3.8 1.5 0.3]
 [4.8 3.  1.4 0.3]]
[[7.9 3.8 6.4 2. ]
 [5.8 2.7 5.1 1.9]
 [6.7 3.  5.2 2.3]
 [5.1 3.8 1.9 0.4]
 [4.7 3.2 1.6 0.2]]
[[6.  2.2 5.  1.5]
 [4.8 3.4 1.6 0.2]
 [7.7 2.6 6.9 2.3]
 [4.6 3.6 1.  0.2]
 [7.2 3.2 6.  1.8]]
[[5.  3.3 1.4 0.2]
 [6.6 3.  4.4 1.4]
 [6.1 2.8 4.  1.3]
 [5.  3.2 1.2 0.2]
 [7.  3.2 4.7 1.4]]
[[6.  3.  4.8 1.8]
 [7.4 2.8 6.1 1.9]
 [5.8 2.7 5.1 1.9]
 [6.2 3.4 5.4 2.3]
 [5.  2.  3.5 1. ]]
[[5.6 2.5 3.9 1.1]
 [6.7 3.1 5.6 2.4]
 [6.3 2.5 5.  1.9]
 [6.4 3.1 5.5 1.8]
 [6.2 2.2 4.5 1.5]]
[[7.3 2.9 6.3 1.8]
 [4.4 3.  1.3 0.2]
 [7.2 3.6 6.1 2.5]
 [6.5 3.  5.5 1.8]
 [5.  3.4 1.5 0.2]]
[[4.7 3.2 1.3 0.2]
 [6.6 2.9 4.6 1.3]
 [5.5 3.5 1.3 0.2]
 [7.7 3.  6.1 2.3]
 [6.1 3.  4.9 1.8]]
[[4.9 3.1 1.5 0.1]
 [5.5 2.4 3.8 1.1]
 [5.7 2.9 4.2 1.3]
 [6.  2.9 4.5 1.5]
 [6.4 2.7 5.3 1.9]]
[[5.4 3.7 1.5 0.2]
 [6.1 2.9 4.7 1.4]
 [

 [7.  3.2 4.7 1.4]]
[[6.  3.  4.8 1.8]
 [7.4 2.8 6.1 1.9]
 [5.8 2.7 5.1 1.9]
 [6.2 3.4 5.4 2.3]
 [5.  2.  3.5 1. ]]
[[5.6 2.5 3.9 1.1]
 [6.7 3.1 5.6 2.4]
 [6.3 2.5 5.  1.9]
 [6.4 3.1 5.5 1.8]
 [6.2 2.2 4.5 1.5]]
[[7.3 2.9 6.3 1.8]
 [4.4 3.  1.3 0.2]
 [7.2 3.6 6.1 2.5]
 [6.5 3.  5.5 1.8]
 [5.  3.4 1.5 0.2]]
[[4.7 3.2 1.3 0.2]
 [6.6 2.9 4.6 1.3]
 [5.5 3.5 1.3 0.2]
 [7.7 3.  6.1 2.3]
 [6.1 3.  4.9 1.8]]
[[4.9 3.1 1.5 0.1]
 [5.5 2.4 3.8 1.1]
 [5.7 2.9 4.2 1.3]
 [6.  2.9 4.5 1.5]
 [6.4 2.7 5.3 1.9]]
[[5.4 3.7 1.5 0.2]
 [6.1 2.9 4.7 1.4]
 [6.5 2.8 4.6 1.5]
 [5.6 2.7 4.2 1.3]
 [6.3 3.4 5.6 2.4]]
[[4.9 3.1 1.5 0.1]
 [6.8 2.8 4.8 1.4]
 [5.7 2.8 4.5 1.3]
 [6.  2.7 5.1 1.6]
 [5.  3.5 1.3 0.3]]
[[6.5 3.  5.2 2. ]
 [6.1 2.8 4.7 1.2]
 [5.1 3.5 1.4 0.3]
 [4.6 3.1 1.5 0.2]
 [6.5 3.  5.8 2.2]]
[[4.6 3.4 1.4 0.3]
 [4.6 3.2 1.4 0.2]
 [7.7 2.8 6.7 2. ]
 [5.9 3.2 4.8 1.8]
 [5.1 3.8 1.6 0.2]]
[[4.9 3.  1.4 0.2]
 [4.9 2.4 3.3 1. ]
 [4.5 2.3 1.3 0.3]
 [5.8 2.7 4.1 1. ]
 [5.  3.4 1.6 0.4]]
[[5.2 3.4 1.4 0.2]
 

[[6.4 2.8 5.6 2.2]
 [5.  2.3 3.3 1. ]
 [4.9 2.5 4.5 1.7]
 [4.9 3.1 1.5 0.1]
 [5.7 3.8 1.7 0.3]]
[[4.4 3.2 1.3 0.2]
 [5.4 3.4 1.5 0.4]
 [6.9 3.1 5.1 2.3]
 [6.7 3.1 4.4 1.4]
 [5.1 3.7 1.5 0.4]]
[[5.2 2.7 3.9 1.4]
 [6.9 3.1 4.9 1.5]
 [5.8 4.  1.2 0.2]
 [5.4 3.9 1.7 0.4]
 [7.7 3.8 6.7 2.2]]
[[6.3 3.3 4.7 1.6]
 [6.8 3.2 5.9 2.3]
 [7.6 3.  6.6 2.1]
 [6.4 3.2 5.3 2.3]
 [5.7 4.4 1.5 0.4]]
[[6.7 3.3 5.7 2.1]
 [6.4 2.8 5.6 2.1]
 [5.4 3.9 1.3 0.4]
 [6.1 2.6 5.6 1.4]
 [7.2 3.  5.8 1.6]]
[[5.2 3.5 1.5 0.2]
 [5.8 2.6 4.  1.2]
 [5.9 3.  5.1 1.8]
 [5.4 3.  4.5 1.5]
 [6.7 3.  5.  1.7]]
[[6.3 2.3 4.4 1.3]
 [5.1 2.5 3.  1.1]
 [6.4 3.2 4.5 1.5]
 [6.8 3.  5.5 2.1]
 [6.2 2.8 4.8 1.8]]
[[6.9 3.2 5.7 2.3]
 [6.5 3.2 5.1 2. ]
 [5.8 2.8 5.1 2.4]
 [5.1 3.8 1.5 0.3]
 [4.8 3.  1.4 0.3]]
[[7.9 3.8 6.4 2. ]
 [5.8 2.7 5.1 1.9]
 [6.7 3.  5.2 2.3]
 [5.1 3.8 1.9 0.4]
 [4.7 3.2 1.6 0.2]]
[[6.  2.2 5.  1.5]
 [4.8 3.4 1.6 0.2]
 [7.7 2.6 6.9 2.3]
 [4.6 3.6 1.  0.2]
 [7.2 3.2 6.  1.8]]
[[5.  3.3 1.4 0.2]
 [6.6 3.  4.4 1.4]
 [

[[6.  3.  4.8 1.8]
 [7.4 2.8 6.1 1.9]
 [5.8 2.7 5.1 1.9]
 [6.2 3.4 5.4 2.3]
 [5.  2.  3.5 1. ]]
[[5.6 2.5 3.9 1.1]
 [6.7 3.1 5.6 2.4]
 [6.3 2.5 5.  1.9]
 [6.4 3.1 5.5 1.8]
 [6.2 2.2 4.5 1.5]]
[[7.3 2.9 6.3 1.8]
 [4.4 3.  1.3 0.2]
 [7.2 3.6 6.1 2.5]
 [6.5 3.  5.5 1.8]
 [5.  3.4 1.5 0.2]]
[[4.7 3.2 1.3 0.2]
 [6.6 2.9 4.6 1.3]
 [5.5 3.5 1.3 0.2]
 [7.7 3.  6.1 2.3]
 [6.1 3.  4.9 1.8]]
[[4.9 3.1 1.5 0.1]
 [5.5 2.4 3.8 1.1]
 [5.7 2.9 4.2 1.3]
 [6.  2.9 4.5 1.5]
 [6.4 2.7 5.3 1.9]]
[[5.4 3.7 1.5 0.2]
 [6.1 2.9 4.7 1.4]
 [6.5 2.8 4.6 1.5]
 [5.6 2.7 4.2 1.3]
 [6.3 3.4 5.6 2.4]]
[[4.9 3.1 1.5 0.1]
 [6.8 2.8 4.8 1.4]
 [5.7 2.8 4.5 1.3]
 [6.  2.7 5.1 1.6]
 [5.  3.5 1.3 0.3]]
[[6.5 3.  5.2 2. ]
 [6.1 2.8 4.7 1.2]
 [5.1 3.5 1.4 0.3]
 [4.6 3.1 1.5 0.2]
 [6.5 3.  5.8 2.2]]
[[4.6 3.4 1.4 0.3]
 [4.6 3.2 1.4 0.2]
 [7.7 2.8 6.7 2. ]
 [5.9 3.2 4.8 1.8]
 [5.1 3.8 1.6 0.2]]
[[4.9 3.  1.4 0.2]
 [4.9 2.4 3.3 1. ]
 [4.5 2.3 1.3 0.3]
 [5.8 2.7 4.1 1. ]
 [5.  3.4 1.6 0.4]]
[[5.2 3.4 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [

[[4.9 3.1 1.5 0.1]
 [6.8 2.8 4.8 1.4]
 [5.7 2.8 4.5 1.3]
 [6.  2.7 5.1 1.6]
 [5.  3.5 1.3 0.3]]
[[6.5 3.  5.2 2. ]
 [6.1 2.8 4.7 1.2]
 [5.1 3.5 1.4 0.3]
 [4.6 3.1 1.5 0.2]
 [6.5 3.  5.8 2.2]]
[[4.6 3.4 1.4 0.3]
 [4.6 3.2 1.4 0.2]
 [7.7 2.8 6.7 2. ]
 [5.9 3.2 4.8 1.8]
 [5.1 3.8 1.6 0.2]]
[[4.9 3.  1.4 0.2]
 [4.9 2.4 3.3 1. ]
 [4.5 2.3 1.3 0.3]
 [5.8 2.7 4.1 1. ]
 [5.  3.4 1.6 0.4]]
[[5.2 3.4 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.6 2.9 3.6 1.3]
 [4.8 3.1 1.6 0.2]]
[[6.3 2.7 4.9 1.8]
 [5.7 2.8 4.1 1.3]
 [5.  3.  1.6 0.2]
 [6.3 3.3 6.  2.5]
 [5.  3.5 1.6 0.6]]
[[5.5 2.6 4.4 1.2]
 [5.7 3.  4.2 1.2]
 [4.4 2.9 1.4 0.2]
 [4.8 3.  1.4 0.1]
 [5.5 2.4 3.7 1. ]]
Epoch  8
[[6.4 2.8 5.6 2.2]
 [5.  2.3 3.3 1. ]
 [4.9 2.5 4.5 1.7]
 [4.9 3.1 1.5 0.1]
 [5.7 3.8 1.7 0.3]]
[[4.4 3.2 1.3 0.2]
 [5.4 3.4 1.5 0.4]
 [6.9 3.1 5.1 2.3]
 [6.7 3.1 4.4 1.4]
 [5.1 3.7 1.5 0.4]]
[[5.2 2.7 3.9 1.4]
 [6.9 3.1 4.9 1.5]
 [5.8 4.  1.2 0.2]
 [5.4 3.9 1.7 0.4]
 [7.7 3.8 6.7 2.2]]
[[6.3 3.3 4.7 1.6]
 [6.8 3.2 5.